## if include sensor_id then can fit quite a good model - but this would be a model that would only be able to predict at those locations

In [1]:
import os
import pandas as pd
from time import time
# from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
# import numpy as np
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor 
# import xgboost as xgb
from sklearn.pipeline import Pipeline
# import folium
# import branca.colormap as cm
# from eli5.sklearn import PermutationImportance
# import joblib
# import os
# import psutil
# import geopy.distance
import sys

sys.path.append('../')
from Functions import *

/opt/homebrew/anaconda3/envs/py310/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [2]:
buffer_size_m = 400
input_csv =f"../../Cleaned_data/FormattedDataForModelling/formatted_data_for_modelling_allsensors_{buffer_size_m}_outlierremovaleachsensor.csv"

## Run models with cross-validation

### Define the error metrics for the cross-validation to return, and the parameters of the cross validation

In [3]:
error_metrics = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error']
cv_parameters = KFold(n_splits=10, random_state=1, shuffle=True)

In [4]:
rf_model_pipeline = Pipeline(steps=[['scaler',StandardScaler()],['rf_regressor', RandomForestRegressor(random_state = 1, n_jobs = 10)]])

### Prepare data for modelling

In [5]:
Xfull, Yfull, data_time_columns, index2019 = prepare_x_y_data(input_csv)

/Users/geonsm/gp/footfall/MelbourneAnalysis/3. Modelling/ModellingExtras/../Functions.py:28: DtypeWarning: Columns (4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_csv, index_col = False)


### Cut off data post-Covid

In [6]:
Xfull= Xfull[0:index2019]
Yfull= Yfull[0:index2019]
data_time_columns = data_time_columns[0:index2019] # end of 2019

### Add sensor ID

In [7]:
sensor_ids = pd.read_csv(input_csv)['sensor_id']
sensor_ids= sensor_ids[0:index2019]
Xfull['sensor_id'] = sensor_ids

/var/folders/bx/3j5z8qc50wl1tcff6xvbf2zh0000gs/T/ipykernel_94975/983852409.py:1: DtypeWarning: Columns (4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  sensor_ids = pd.read_csv(input_csv)['sensor_id']


### Choose which month_num and weekday_num option to include


In [8]:
# If using the dummy variables
# Xfull.drop(['Cos_month_num', 'Sin_month_num', 'Cos_weekday_num', 'Sin_weekday_num'], axis=1)
# If using the cyclical variables
#Xfull.drop(['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday',
#       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
#       'month_8', 'month_9', 'month_10', 'month_11', 'month_12'], axis=1, inplace = True)

#In NM version, using the cyclical variables, the dummy variables are not included so do not need to be dropped. There are just a few to be removed
# Also 'new_sensor' made it through for some reason
Xfull.drop(['year', 'day', 'month', 'new_sensor'], axis=1, inplace = True)

### Remove spatial features

In [9]:
spatial_cols = ['betweenness', 'lights',  'memorials', 'trees','bus-stops', 'tram-stops', 'metro-stations',
                'taxi-ranks', 'big-car-parks', 'street_inf_Bicycle Rails', 'street_inf_Bollard','street_inf_Drinking Fountain',
                'street_inf_Floral Crate/Planter Box','street_inf_Horse Trough', 'street_inf_Information Pillar',
                'street_inf_Litter Bin', 'street_inf_Seat', 'street_inf_Tree Guard','landmarks_Community Use',
                'landmarks_Mixed Use','landmarks_Place Of Assembly', 'landmarks_Place of Worship', 'landmarks_Retail',
                'landmarks_Transport', 'landmarks_Education Centre','landmarks_Leisure/Recreation', 'landmarks_Office',
                'street_inf_Barbeque', 'street_inf_Hoop', 'street_inf_Picnic Setting', 'landmarks_Specialist Residential Accommodation',
                'landmarks_Vacant Land', 'landmarks_Purpose Built','landmarks_Health Services', 'avg_n_floors', 'buildings_Community Use',
                'buildings_Education', 'buildings_Entertainment', 'buildings_Events','buildings_Hospital/Clinic', 'buildings_Office', 'buildings_Parking',
                'buildings_Public Display Area', 'buildings_Residential','buildings_Retail', 'buildings_Storage', 'buildings_Unoccupied',
                'buildings_Working', 'buildings_Transport',
                'mean_age', 'mean_income', 'mean_school_year']

In [10]:
columns_to_save = Xfull[spatial_cols]

In [11]:
Xfull.drop(spatial_cols, axis=1, inplace = True)

In [12]:
Xfull

,Temp,Humidity,Pressure,Rain,WindSpeed,Rainfall amount (millimetres),public_holiday,school_holiday,transport_stops,Sin_time,Cos_time,Sin_month_num,Cos_month_num,Sin_weekday_num,Cos_weekday_num,distance_from_centre,random,random_cat,sensor_id
0,15.0,77.0,1013.0,0,14.0,0.0,0,0,1,0.000000e+00,1.0000,-2.449000e-16,1.0,0.9749,-0.2225,0.2928,0.613843,2,1
1,15.0,77.0,1014.0,0,12.0,0.0,0,0,1,2.698000e-01,0.9629,-2.449000e-16,1.0,0.9749,-0.2225,0.2928,0.498930,0,1
2,14.0,82.0,1013.0,0,9.0,0.0,0,0,1,5.196000e-01,0.8544,-2.449000e-16,1.0,0.9749,-0.2225,0.2928,0.523841,2,1
3,14.0,79.5,1013.0,0,9.0,0.0,0,0,1,7.308000e-01,0.6826,-2.449000e-16,1.0,0.9749,-0.2225,0.2928,0.419273,2,1
4,12.5,85.0,1014.0,0,7.0,0.0,0,0,1,8.879000e-01,0.4601,-2.449000e-16,1.0,0.9749,-0.2225,0.2928,0.140750,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680323,18.5,43.0,1014.0,0,17.5,0.0,0,0,6,-8.879000e-01,0.4601,-2.449000e-16,1.0,0.9749,-0.2225,0.2468,0.511012,0,62
2680324,17.5,52.0,1014.0,0,15.5,0.0,0,0,6,-7.308000e-01,0.6826,-2.449000e-16,1.0,0.9749,-0.2225,0.2468,0.365936,1,62
2680325,16.0,57.0,1014.0,0,12.0,0.0,0,0,6,-5.196000e-01,0.8544,-2.449000e-16,1.0,0.9749,-0.2225,0.2468,0.352883,0,62
2680326,15.5,63.5,1015.0,0,8.5,0.0,0,0,6,-2.698000e-01,0.9629,-2.449000e-16,1.0,0.9749,-0.2225,0.2468,0.746033,1,62


In [13]:
# Keep only the sensor ID
Xfull_sensorid = Xfull.loc[:, Xfull.columns != 'distance_from_centre']
# Keep only the distance from the centre
Xfull_distance_from_centre = Xfull.loc[:, Xfull.columns != 'sensor_id']
# Keep no spatial variables
Xfull_nospatialvariables = Xfull.loc[:, ~Xfull.columns.isin(['sensor_id', 'distance_from_centre'])]
# Version with spatial variables
Xfull = pd.concat([Xfull_distance_from_centre, columns_to_save], axis=1)

In [14]:
# Dataframe to store the scores for all the models
error_metric_scores = pd.DataFrame()

Xfulls = [Xfull,Xfull_nospatialvariables, Xfull_sensorid, Xfull_distance_from_centre]
version = ['Original', 'No Spatial Features', 'Sensor ID', 'Distance From Centre']
assert len(Xfulls) == len(version)

In [24]:
# Dataframe to store the scores for all the models
error_metric_scores = pd.DataFrame()

os.makedirs("../Results/CV/ComparingSpatialFeatures", exist_ok=True)

for num in range(0,len(Xfulls)):
    # Get the right Xfull from list
    Xfull=Xfulls[num]
    print(f"Starting {version[num]}...")
    # Use cross_validate to return the error scores associated with this model and this data
    start = time()
    model_output = cross_validate(rf_model_pipeline, Xfull, Yfull, cv=cv_parameters, scoring=error_metrics, error_score="raise")
    end = time()
    print('\tRan in {} minutes'.format(round((end - start)/60),2))

    # Formulate the different error scores into a dataframe
    error_metrics_df =pd.DataFrame({'mae': round(abs(model_output['test_neg_mean_absolute_error'].mean()),2), 
                  'mape': round(abs(model_output['test_neg_mean_absolute_percentage_error'].mean()),2),
                  'r2': round(abs(model_output['test_r2'].mean()),2), 
                  'rmse': round(abs(model_output['test_neg_root_mean_squared_error'].mean()),2)},
                 index =[version[num]])

    # Add evaluation metric scores for this model to the dataframe containing the metrics for each model
    error_metric_scores = pd.concat([error_metric_scores, error_metrics_df])

    # Save error scores for this distance to file
#error_metrics_df.to_csv('Results/CV/ComparingModels/{}_{}m_error_metric_scores.csv'.format(model_name,buffer_size_m),index=False)    

# Save dataframes of error metrics for each buffer distance 
error_metric_scores.to_csv('../Results/CV/ComparingSpatialFeatures/comparingmodels_error_metric_scores.csv')

Starting No Spatial Features...
	Ran in 29 minutes
Starting Sensor ID...
	Ran in 30 minutes
Starting Distance From Centre...
	Ran in 29 minutes


In [25]:
error_metric_scores


,mae,mape,r2,rmse
Original,79.33,8.100920e+14,0.96,171.67
No Spatial Features,317.41,1.851024e+15,0.57,529.32
Sensor ID,101.42,9.491138e+14,0.94,206.22
Distance From Centre,101.88,9.415121e+14,0.93,207.07
